# Week 6. 쉽고 강력한 데이터프레임 라이브러리 - Pandas (3)


<br><br><br>
# 시계열 자료 다루기
----------------------------------

pandas에서 일반적인 테이블 형태의 자료와 시계열 자료의 차이점은 인덱스(Index)에 있다.

일반적인 테이블 형태의 자료는 임의의 값을 인덱스로 가질 수 있지만 시계열 자료는`DatetimeIndex`: 타임스탬프 클래스를 인덱스로 가진다.

<br><br><br>


# DatetimeIndex
----------------------------------
`DatetimeIndex`는 특정한 순간에 기록된 타임스탬프(timestamp) 형식의 시계열 자료를 다루기 위한 인덱스이다. 타임스탬프 인덱스는 반드시 일정한 간격으로 자료가 있어야 한다는 조건은 없다.

DatetimeIndex 타입의 인덱스는 보통 다음 방법으로 생성한다.
- `pd.to_datetime` 함수
- `pd.date_range` 함수


## to_datetime

In [1]:
import pandas as pd
import numpy as np

In [12]:
date_str = ["2016, 1, 1", "2016, 1, 4", "2016, 1, 5", "2016, 1, 6"]
idx = pd.to_datetime(date_str)
idx

DatetimeIndex(['2016-01-01', '2016-01-04', '2016-01-05', '2016-01-06'], dtype='datetime64[ns]', freq=None)

In [13]:
np.random.seed(0)
s = pd.Series(np.random.randn(4), index=idx)
s

2016-01-01    1.764052
2016-01-04    0.400157
2016-01-05    0.978738
2016-01-06    2.240893
dtype: float64

## date_range

시작일과 종료일 또는 시작일과 기간을 입력하면 범위 내의 날짜 및 시간 인덱스 생성
`period` 인수 : 시작일로부터 몇일 동안
`freq` 인수
   - `T`: 시간별
   - `D`: 일별
   - `B`: Business day
   - `BMS`: 매월 근무 날짜로 초일
   - `MS`: 매월 초일
   - `M`: 매월 말일
   - `W-MON`: 매주 월요일
   - http://pandas.pydata.org/pandas-docs/stable/timeseries.html#dateoffset-objects


In [2]:
pd.date_range("2016-4-1", "2016-4-30", freq='B')

DatetimeIndex(['2016-04-01', '2016-04-04', '2016-04-05', '2016-04-06',
               '2016-04-07', '2016-04-08', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-18',
               '2016-04-19', '2016-04-20', '2016-04-21', '2016-04-22',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29'],
              dtype='datetime64[ns]', freq='B')

## shift 연산 - 날짜 이동

In [19]:
ts = pd.Series(np.random.randn(4), index=pd.date_range("2000-1-1", periods=4, freq="M"))
ts

2000-01-31   -0.103219
2000-02-29    0.410599
2000-03-31    0.144044
2000-04-30    1.454274
Freq: M, dtype: float64

In [20]:
ts.shift(1)

2000-01-31         NaN
2000-02-29   -0.103219
2000-03-31    0.410599
2000-04-30    0.144044
Freq: M, dtype: float64

In [21]:
ts.shift(-1)

2000-01-31    0.410599
2000-02-29    0.144044
2000-03-31    1.454274
2000-04-30         NaN
Freq: M, dtype: float64

In [22]:
ts.shift(1, freq="M")

2000-02-29   -0.103219
2000-03-31    0.410599
2000-04-30    0.144044
2000-05-31    1.454274
Freq: M, dtype: float64

## 리샘플링 (Resampling)
- `up-sampling` : 구간이 작아지는 경우
- `down-sampling` : 구간이 커지는 경우

In [31]:
ts = pd.Series(np.random.randn(100), index=pd.date_range("2000-1-1", periods=100, freq="D"))
ts.head(20)

2000-01-01    0.605120
2000-01-02    0.895556
2000-01-03   -0.131909
2000-01-04    0.404762
2000-01-05    0.223844
2000-01-06    0.329623
2000-01-07    1.285984
2000-01-08   -1.506998
2000-01-09    0.676461
2000-01-10   -0.382009
2000-01-11   -0.224259
2000-01-12   -0.302250
2000-01-13   -0.375147
2000-01-14   -1.226196
2000-01-15    0.183339
2000-01-16    1.670943
2000-01-17   -0.056133
2000-01-18   -0.001385
2000-01-19   -0.687299
2000-01-20   -0.117475
Freq: D, dtype: float64

In [24]:
ts.resample('W').mean()

2000-01-02    0.441356
2000-01-09   -0.146794
2000-01-16    0.207122
2000-01-23    0.045536
2000-01-30   -0.081312
2000-02-06   -0.398918
2000-02-13   -0.287513
2000-02-20   -0.353319
2000-02-27   -0.567629
2000-03-05   -0.055802
2000-03-12   -0.309877
2000-03-19    0.566217
2000-03-26    0.752381
2000-04-02   -0.058560
2000-04-09    0.994230
Freq: W-SUN, dtype: float64

In [32]:
# Down sampling
ts.resample('MS').first()

2000-01-01    0.605120
2000-02-01   -0.996213
2000-03-01   -1.945703
2000-04-01    1.094638
Freq: MS, dtype: float64

In [33]:
ts = pd.Series(np.random.randn(60), index=pd.date_range("2000-1-1", periods=60, freq="T"))
ts.head(20)

2000-01-01 00:00:00   -0.696327
2000-01-01 00:01:00   -0.290397
2000-01-01 00:02:00    1.327783
2000-01-01 00:03:00   -0.101281
2000-01-01 00:04:00   -0.803141
2000-01-01 00:05:00   -0.464338
2000-01-01 00:06:00    1.021791
2000-01-01 00:07:00   -0.552541
2000-01-01 00:08:00   -0.386871
2000-01-01 00:09:00   -0.510293
2000-01-01 00:10:00    0.183925
2000-01-01 00:11:00   -0.385490
2000-01-01 00:12:00   -1.601836
2000-01-01 00:13:00   -0.887181
2000-01-01 00:14:00   -0.932789
2000-01-01 00:15:00    1.243319
2000-01-01 00:16:00    0.812674
2000-01-01 00:17:00    0.587259
2000-01-01 00:18:00   -0.505358
2000-01-01 00:19:00   -0.815792
Freq: T, dtype: float64

In [34]:
ts.resample('10min').sum()

2000-01-01 00:00:00   -1.455615
2000-01-01 00:10:00   -2.301267
2000-01-01 00:20:00   -5.438233
2000-01-01 00:30:00   -0.267091
2000-01-01 00:40:00    1.201147
2000-01-01 00:50:00    1.440131
Freq: 10T, dtype: float64

In [37]:
ts.resample('10min', closed="right").sum()

1999-12-31 23:50:00   -0.696327
2000-01-01 00:00:00   -0.575363
2000-01-01 00:10:00   -2.992710
2000-01-01 00:20:00   -3.175730
2000-01-01 00:30:00   -1.573881
2000-01-01 00:40:00    1.476052
2000-01-01 00:50:00    0.717031
Freq: 10T, dtype: float64

In [38]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,-0.696327,1.327783,-0.803141,-0.803141
2000-01-01 00:05:00,-0.464338,1.021791,-0.552541,-0.510293
2000-01-01 00:10:00,0.183925,0.183925,-1.601836,-0.932789
2000-01-01 00:15:00,1.243319,1.243319,-0.815792,-0.815792
2000-01-01 00:20:00,-0.507518,2.497200,-2.245322,0.564009
2000-01-01 00:25:00,-1.284552,-0.104343,-1.284552,-1.140196
2000-01-01 00:30:00,1.754986,1.754986,-0.765702,0.010349
2000-01-01 00:35:00,0.720034,0.772695,-1.824257,-1.661598
2000-01-01 00:40:00,0.448195,1.696182,-0.014858,0.670570
2000-01-01 00:45:00,-0.707506,0.265688,-1.566995,0.265688
